In [299]:
#!pip install pmdarima
# !pip install openpyxl
# !pip install ast
"""
Importing all libraries to execute code
"""


# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Basic imports
import os
import time
import json
import ast
import pickle
import itertools
import numpy as np
import pandas as pd
from datetime import datetime

# Data visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.dates import date2num
from matplotlib.ticker import FuncFormatter
import seaborn as sns
%matplotlib inline

# Statistics
from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

# Prophet (Time Series Forecasting)
import prophet as Prophet

# Time Series Analysis
from pmdarima import auto_arima

# Machine Learning: XGBoost
import xgboost as xgb
from xgboost.sklearn import XGBRegressor  # Wrapper
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Excel File Manipulation
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl

# Data Profiling
from dataprofiler import Profiler, Data
from collections import defaultdict

# Custom Function (from original code block)
def millions(x, pos):
    return f'{x * 1e-6:.1f}M'
qualitative_colors = [
    "#1f77b4",  # Blue
    "#ff7f0e",  # Orange
    "#2ca02c",  # Green
    "#d62728",  # Red
    "#9467bd",  # Purple
    "#8c564b",  # Brown
    "#e377c2",  # Pink
    "#7f7f7f",  # Gray
    "#bcbd22",  # Olive
    "#17becf"   # Cyan
]

In [300]:
# importing train data to learn
train = pd.read_csv("train.csv", 
                    parse_dates = True, index_col = 'Date')

# additional store data
store = pd.read_csv("store.csv")
# time series as indexes
# dataTypeAnalyze = Data("train.csv")
# profile = Profiler(dataTypeAnalyze)

# human_readable_report = profile.report(report_options={"output_format":"pretty"})

# print(json.dumps(human_readable_report, indent=4))
display(train)
display(store)



# for column in train.columns:
#     unique_values = train[column].unique()
#     print(f"Unique values in column '{column}': {unique_values}")
# print(train.shape)

# for column in store.columns:
#     unique_values = store[column].unique()
#     print(f"Unique values in column '{column}': {unique_values}")
# print(store.shape)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1
2015-07-31,2,5,6064,625,1,1,0,1
2015-07-31,3,5,8314,821,1,1,0,1
2015-07-31,4,5,13995,1498,1,1,0,1
2015-07-31,5,5,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...
2013-01-01,1111,2,0,0,0,0,a,1
2013-01-01,1112,2,0,0,0,0,a,1
2013-01-01,1113,2,0,0,0,0,a,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [301]:
print("The data runs over ", train.index.nunique(), "days")
print("The unique number of stores ", train['Store'].nunique())


# data extraction
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekOfYear'] = train.index.isocalendar().week 

# adding new variable
train['SalePerCustomer'] = train['Sales']/train['Customers']

The data runs over  942 days
The unique number of stores  1115


In [302]:
# train_store = pd.merge(train, store, how = 'inner', on = 'Store')
# train_store.set_index(train.index, inplace=True)
# ... existing code ...
train_store = pd.merge(train.reset_index(), store, how='inner', on='Store')
train_store.set_index('Date', inplace=True)
# ... existing code ...
train_store[train_store['Store']==1114].to_clipboard()

In [303]:

train_a = train_store[train_store['StoreType']=='a']
print(train_a)

train_a = train_a.sort_values(by='Date',ascending=True)
#train_a['Date'] = pd.to_datetime(train_a.index, format='%Y-%m-%d')  # Ensure 'Date' is in YYYY-MM-DD format
train_a[train_a['Store']==1114].to_clipboard()
stores = train_a['Store'].unique().tolist()




            Store  DayOfWeek  Sales  Customers  Open  Promo StateHoliday  \
Date                                                                       
2015-07-31      2          5   6064        625     1      1            0   
2015-07-30      2          4   5567        601     1      1            0   
2015-07-29      2          3   6402        727     1      1            0   
2015-07-28      2          2   5671        646     1      1            0   
2015-07-27      2          1   6627        638     1      1            0   
...           ...        ...    ...        ...   ...    ...          ...   
2013-01-05   1114          6  18856       3065     1      0            0   
2013-01-04   1114          5  18371       3036     1      0            0   
2013-01-03   1114          4  18463       3211     1      0            0   
2013-01-02   1114          3  20642       3401     1      0            0   
2013-01-01   1114          2      0          0     0      0            a   

           

In [304]:
# count=0
promos = {}
promoslist = []
for store in stores:
    store_df = train_a[train_a['Store']==store].copy()
    store_df = store_df.sort_values(by='Date',ascending=True)
    # pd.set_option('display.max_rows', 20)
    # display(store_df)
    promos[store] = store_df['Promo'].to_list()
    if store == 1114:
        store_df.to_clipboard()
        print(store_df)
    # promos[store] = train_a[train_a['Store']==store]['Promo'].to_list()
    # promoslist.append(train_a[train_a['Store']==store]['Promo'].to_list())
    # train_a[train_a['Store'==store]]

# print(type(train_a[train_a['Store']==2]['Promo'].to_list()))
# print(train_a[train_a['Store']==2]['Promo'].to_list())

            Store  DayOfWeek  Sales  Customers  Open  Promo StateHoliday  \
Date                                                                       
2013-01-01   1114          2      0          0     0      0            a   
2013-01-02   1114          3  20642       3401     1      0            0   
2013-01-03   1114          4  18463       3211     1      0            0   
2013-01-04   1114          5  18371       3036     1      0            0   
2013-01-05   1114          6  18856       3065     1      0            0   
...           ...        ...    ...        ...   ...    ...          ...   
2015-07-27   1114          1  26720       3468     1      1            0   
2015-07-28   1114          2  25518       3456     1      1            0   
2015-07-29   1114          3  25840       3545     1      1            0   
2015-07-30   1114          4  24395       3251     1      1            0   
2015-07-31   1114          5  27508       3745     1      1            0   

           